In [7]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 3.0 MB/s eta 0:00:003.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 3.2 MB/s eta 0:00:004.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.8 MB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 3.5 MB/s eta 0:00:00m eta 0:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# read train dataset
train = pd.read_csv('train.csv')

# split train dataset into training and validation sets
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)

# select relevant features
features = [
    'IsHoliday',
    'Total_MarkDown',
    'Month',
    'Fuel_Price',
    'Store',
    'Week',
    'Size',
    'mean',
    'min',
    'CPI',
    'Unemployment',
    'Dept',
    'median',
    'max',
    'Year',
#     'Type'
]

# preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[features])
y_train = np.array(train_data['Weekly_Sales'])
X_val = scaler.transform(val_data[features])
y_val = np.array(val_data['Weekly_Sales'])

# create neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# compile model
model.compile(optimizer='adam', loss='mse')

# train model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))

# evaluate model's performance on validation set
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print('Validation set RMSE:', rmse)

# read test dataset
test = pd.read_csv('test.csv')

# preprocess test dataset
X_test = scaler.transform(test[features])

# make predictions on test dataset
y_test = model.predict(X_test)

# save predictions to file
test['Weekly_Sales_Pred'] = y_test
test[['id', 'Weekly_Sales_Pred']].to_csv('predictions.csv', index=False)


Epoch 1/10
1638/1638 [==============================] - 3s 1ms/step - loss: 274829152.0000 - val_loss: 51838804.0000
Epoch 2/10
1638/1638 [==============================] - 2s 1ms/step - loss: 23299154.0000 - val_loss: 19727076.0000
Epoch 3/10
1638/1638 [==============================] - 2s 1ms/step - loss: 19280704.0000 - val_loss: 19655500.0000
Epoch 4/10
1638/1638 [==============================] - 2s 1ms/step - loss: 19245854.0000 - val_loss: 19634374.0000
Epoch 5/10
1638/1638 [==============================] - 2s 1ms/step - loss: 19234498.0000 - val_loss: 19621084.0000
Epoch 6/10
1638/1638 [==============================] - 2s 1ms/step - loss: 19220558.0000 - val_loss: 19605580.0000
Epoch 7/10
1638/1638 [==============================] - 2s 1ms/step - loss: 19203682.0000 - val_loss: 19590198.0000
Epoch 8/10
1638/1638 [==============================] - 2s 1ms/step - loss: 19187294.0000 - val_loss: 19571294.0000
Epoch 9/10
1638/1638 [==============================] - 2s 1ms/step - l

In [13]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# read train dataset
train = pd.read_csv('train.csv')

# split train dataset into training and validation sets
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)

# select relevant features
# features = ['IsHoliday', 'Total_MarkDown', 'Month', 'Fuel_Price', 'Store', 'Week', 'Size', 'mean', 'min', 'CPI', 'Unemployment', 'Dept', 'median', 'max', 'Year', 'Type']
features = [
#     'IsHoliday',
#     'Total_MarkDown',
    'Month',
#     'Fuel_Price',
    'Store',
    'Week',
    'Size',
    'mean',
    'min',
#     'CPI',
#     'Unemployment',
#     'Dept',
    'median',
    'max',
    'std'
#     'Year',
#     'Type'
]
# preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[features])
y_train = np.array(train_data['Weekly_Sales'])
X_val = scaler.transform(val_data[features])
y_val = np.array(val_data['Weekly_Sales'])

# create neural network model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# compile model
model.compile(optimizer='adam', loss='mse')

# set up early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# train model
model.fit(X_train, y_train, epochs=50, batch_size=256, validation_data=(X_val, y_val), callbacks=[early_stop])

# evaluate model's performance on validation set
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print('Validation set RMSE:', rmse)

# read test dataset
test = pd.read_csv('test.csv')

# preprocess test dataset
X_test = scaler.transform(test[features])

# make predictions on test dataset
y_test = model.predict(X_test)

# save predictions to file
test['Weekly_Sales_Pred'] = y_test
test[['id', 'Weekly_Sales_Pred']].to_csv('predictions.csv', index=False)


Epoch 1/50
819/819 [==============================] - 3s 3ms/step - loss: 81418056.0000 - val_loss: 19185526.0000
Epoch 2/50
819/819 [==============================] - 3s 4ms/step - loss: 19757956.0000 - val_loss: 18277698.0000
Epoch 3/50
819/819 [==============================] - 3s 3ms/step - loss: 19417822.0000 - val_loss: 18159404.0000
Epoch 4/50
819/819 [==============================] - 3s 3ms/step - loss: 19257598.0000 - val_loss: 18040584.0000
Epoch 5/50
819/819 [==============================] - 3s 3ms/step - loss: 19200092.0000 - val_loss: 18018018.0000
Epoch 6/50
819/819 [==============================] - 3s 3ms/step - loss: 19107726.0000 - val_loss: 17972372.0000
Epoch 7/50
819/819 [==============================] - 3s 3ms/step - loss: 19139222.0000 - val_loss: 17896496.0000
Epoch 8/50
819/819 [==============================] - 3s 3ms/step - loss: 19096304.0000 - val_loss: 17863800.0000
Epoch 9/50
819/819 [==============================] - 3s 4ms/step - loss: 19031540.0000 